# Toxic Comment Heterogeneous GNN Training Notebook




## 1. Load Context Configuration (Parse context.md)


In [ ]:
import re, json, os, textwrap
from pathlib import Path
import pandas as pd

CONTEXT_FILE = Path('context.md')
assert CONTEXT_FILE.exists(), 'context.md not found in workspace.'
raw = CONTEXT_FILE.read_text(encoding='utf-8')

# heuristic extraction of CSV path
m = re.search(r'youtube_comments_with_toxicity_(\\d+_\\d+)\\.csv', raw)
if m:
    csv_candidates = list(Path('Notebooks').glob(f'youtube_comments_with_toxicity_{m.group(1)}.csv'))
else:
    csv_candidates = list(Path('Notebooks').glob('youtube_comments_with_toxicity_*.csv'))
DATA_CSV = csv_candidates[0] if csv_candidates else None

config = {
    'data_csv': str(DATA_CSV) if DATA_CSV else None,
    'user_col': 'AuthorChannelID',
    'comment_col': 'CommentText',
    'comment_id_col': 'CommentID',
    'parent_col': 'ParentCommentID',
    'label_col': 'ToxicLabel',
    'score_col': 'ToxicScore',
    'binary_label_col': 'ToxicBinary',
    'embedding_model': 'all-MiniLM-L6-v2',
    'gnn_hidden': 128,
    'gnn_out_classes': 2,
    'seed': 42,
    'train_val_test_split': [0.8,0.1,0.1],
    'primary_metric': 'f1',
    'epochs': 30,
    'lr': 1e-3,
    'weight_decay': 1e-5,
    'early_stopping_patience': 5,
    'batch_size_node_loader': 1024,
    'neighbors': 10
}
print('CONFIG =>')
print(json.dumps(config, indent=2))

if not DATA_CSV:
    raise FileNotFoundError('Could not locate toxicity CSV. Please place it under Notebooks/.')

df = pd.read_csv(DATA_CSV)
print('Loaded dataframe shape:', df.shape)
df.head(2)

## 2. Environment Setup and Imports

In [ ]:
import numpy as np
import torch, random
from pathlib import Path

REQUIRED = ['pandas','numpy','torch','sklearn','tqdm']
print('Python version OK')
print('Torch:', torch.__version__)

def set_seed(seed:int):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
set_seed(config['seed'])

device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if getattr(torch.backends,'mps',None) and torch.backends.mps.is_available() else 'cpu')
print('Using device:', device)

## 3. Data Loading and Preprocessing

In [ ]:
# Ensure required columns & create binary label
USER_COL=config['user_col']; COMMENT_COL=config['comment_col']; CID_COL=config['comment_id_col']
if 'ToxicBinary' not in df.columns:
    if df['ToxicLabel'].dtype==object:
        df['ToxicBinary']=df['ToxicLabel'].str.lower().str.startswith('toxic').astype(int)
    else:
        df['ToxicBinary']=(df['ToxicScore']>0.7).astype(int)

# Drop rows missing essentials
df = df.dropna(subset=[USER_COL, COMMENT_COL])

print('Rows after cleaning:', len(df))
print('Class balance:', df['ToxicBinary'].value_counts(normalize=True))

df.head(3)

## 4. Text Embeddings

In [ ]:

try:
    from sentence_transformers import SentenceTransformer
    model_name = config['embedding_model']
    sbert = SentenceTransformer(model_name)
    texts = df[COMMENT_COL].astype(str).tolist()
    batch=256; embs=[]
    for i in range(0,len(texts),batch):
        embs.append(sbert.encode(texts[i:i+batch], show_progress_bar=False))
    import numpy as np
    embeddings = np.vstack(embs)
except Exception as e:
    print('Falling back to bag-of-words (hash) embeddings due to error:', e)
    import numpy as np, hashlib
    def hvec(t):
        h = hashlib.md5(t.encode()).hexdigest()
        return np.array([int(h[i:i+4],16)%10000 for i in range(0,16,4)],dtype=float)
    embeddings = np.vstack([hvec(t) for t in df[COMMENT_COL].astype(str)])

print('Embeddings shape:', embeddings.shape)

## 5. Build Graph (HeteroData)

In [ ]:
from torch_geometric.data import HeteroData
import torch
# Reply edges heuristic if parent col missing
if config['parent_col'] in df.columns and df[config['parent_col']].notna().any():
    reply_pairs = df[df[config['parent_col']].notna()][[config['parent_col'], config['comment_id_col']]].values.tolist()
else:
    reply_pairs=[]
    if 'VideoID' in df.columns and 'PublishedAt' in df.columns:
        df_sorted=df.sort_values(['VideoID','PublishedAt'])
        for vid, group in df_sorted.groupby('VideoID'):
            ids=group[config['comment_id_col']].tolist()
            for i in range(1,len(ids)):
                reply_pairs.append((ids[i-1], ids[i]))

comment_ids = df[CID_COL].astype(str).tolist()
comment_idx={cid:i for i,cid in enumerate(comment_ids)}
users = df[USER_COL].astype(str).unique().tolist()
user_idx={u:i for i,u in enumerate(users)}

# user->comment authored edges
authored_edges = [(row[USER_COL], row[CID_COL]) for _,row in df.iterrows()]

# Build features for users
import numpy as np
user_deg={u:0 for u in users}; user_tox={u:[] for u in users}
for _,r in df.iterrows():
    u=r[USER_COL]; user_deg[u]+=1; user_tox[u].append(r['ToxicBinary'])
user_feat=np.vstack([
    [user_deg[u] for u in users],
    [np.mean(user_tox[u]) if user_tox[u] else 0 for u in users]
]).T

hetero = HeteroData()
hetero['comment'].x = torch.tensor(embeddings, dtype=torch.float)
hetero['comment'].y = torch.tensor(df['ToxicBinary'].values, dtype=torch.long)
hetero['user'].x = torch.tensor(user_feat, dtype=torch.float)

src=[user_idx[u] for u,c in authored_edges]
dst=[comment_idx[c] for u,c in authored_edges]
hetero['user','authored','comment'].edge_index = torch.tensor([src,dst])

r_src=[comment_idx[p] for p,c in reply_pairs if p in comment_idx and c in comment_idx]
r_dst=[comment_idx[c] for p,c in reply_pairs if p in comment_idx and c in comment_idx]
hetero['comment','replies_to','comment'].edge_index = torch.tensor([r_src,r_dst])

print(hetero)

## 6. Train/Val/Test Split

In [ ]:
import torch
num_comments = hetero['comment'].num_nodes
perm = torch.randperm(num_comments)
n_train=int(config['train_val_test_split'][0]*num_comments)
n_val=int(config['train_val_test_split'][1]*num_comments)
train_idx=perm[:n_train]; val_idx=perm[n_train:n_train+n_val]; test_idx=perm[n_train+n_val:]
train_mask=torch.zeros(num_comments,dtype=torch.bool); train_mask[train_idx]=True
val_mask=torch.zeros(num_comments,dtype=torch.bool); val_mask[val_idx]=True
test_mask=torch.zeros(num_comments,dtype=torch.bool); test_mask[test_idx]=True
hetero['comment'].train_mask=train_mask
hetero['comment'].val_mask=val_mask
hetero['comment'].test_mask=test_mask
print('Split sizes:', train_mask.sum().item(), val_mask.sum().item(), test_mask.sum().item())

## 7. Model Definition

In [ ]:
from torch_geometric.nn import HeteroConv, SAGEConv, Linear
import torch.nn.functional as F

class HeteroGNN(torch.nn.Module):
    def __init__(self, hidden, out_classes):
        super().__init__()
        self.conv1 = HeteroConv({
            ('user','authored','comment'): SAGEConv((-1,-1), hidden),
            ('comment','replies_to','comment'): SAGEConv((-1,-1), hidden)
        }, aggr='mean')
        self.lin = Linear(hidden, out_classes)
    def forward(self, x_dict, edge_index_dict):
        x_dict = self.conv1(x_dict, edge_index_dict)
        c = x_dict['comment'].relu()
        return self.lin(c)

model = HeteroGNN(config['gnn_hidden'], config['gnn_out_classes']).to(device)
hetero = hetero.to(device)
print(model)

## 8. Training Hyperparameters & Optimizer

In [ ]:
import torch.optim as optim
from sklearn.metrics import f1_score, accuracy_score

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])

best_metric=-1.0
patience=0
best_path = Path('model_best.pt')

## 9. Training Loop with Validation & Early Stopping

In [ ]:
from time import time
history = []
for epoch in range(1, config['epochs']+1):
    model.train()
    optimizer.zero_grad()
    out = model(hetero.x_dict, hetero.edge_index_dict)
    y = hetero['comment'].y
    loss = criterion(out[hetero['comment'].train_mask], y[hetero['comment'].train_mask])
    loss.backward(); optimizer.step()

    # validation
    model.eval()
    with torch.no_grad():
        val_logits = out[hetero['comment'].val_mask]
        val_y = y[hetero['comment'].val_mask]
        preds = val_logits.argmax(dim=1).cpu().numpy()
        val_y_np = val_y.cpu().numpy()
        f1 = f1_score(val_y_np, preds, average='macro')
        acc = accuracy_score(val_y_np, preds)
    history.append({'epoch':epoch,'loss':float(loss.item()),'val_f1':float(f1),'val_acc':float(acc)})
    print(f"Epoch {epoch:02d} | loss {loss.item():.4f} | val_f1 {f1:.4f} | val_acc {acc:.4f}")

    if f1>best_metric:
        best_metric=f1; patience=0
        torch.save({'model_state': model.state_dict(), 'config': config}, best_path)
        print('  ✅ Saved new best model (F1={:.4f})'.format(f1))
    else:
        patience+=1
        if patience>=config['early_stopping_patience']:
            print('Early stopping triggered.')
            break

import pandas as _pd
hist_df = _pd.DataFrame(history)
hist_df.head()

## 10. Test Evaluation (Best Model)

In [ ]:
# Load best and evaluate on test
state = torch.load(best_path, map_location=device)
model.load_state_dict(state['model_state'])
model.eval()
with torch.no_grad():
    logits = model(hetero.x_dict, hetero.edge_index_dict)
    test_logits = logits[hetero['comment'].test_mask]
    test_y = hetero['comment'].y[hetero['comment'].test_mask]
    preds = test_logits.argmax(dim=1).cpu().numpy()
    y_true = test_y.cpu().numpy()
    test_f1 = f1_score(y_true, preds, average='macro')
    test_acc = accuracy_score(y_true, preds)
print(f'Test F1={test_f1:.4f} Acc={test_acc:.4f}')

## 11. Serialize Artifacts (Config + Model)

In [ ]:
import json, hashlib
artifacts_dir=Path('artifacts'); artifacts_dir.mkdir(exist_ok=True)
json.dump(config, open(artifacts_dir/'config.json','w'), indent=2)
# copy model file
import shutil
shutil.copy(best_path, artifacts_dir/'model_best.pt')
# hash
h=hashlib.sha256(open(artifacts_dir/'model_best.pt','rb').read()).hexdigest()
print('Model SHA256:', h)

## 11b. Task A: Reply Edge Classification (Abusive vs Non-Abusive)

In [ ]:
# Prepare edge dataset: label is child comment toxicity
import torch, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# r_src, r_dst are comment indices for reply edges (built earlier)
edge_src = np.array(r_src)
edge_dst = np.array(r_dst)

if edge_src.size == 0:
    print('No reply edges available; skipping edge classification.')
else:
    model.eval()
    with torch.no_grad():
        hidden_dict = model.conv1(hetero.x_dict, hetero.edge_index_dict)
        hidden_comment = hidden_dict['comment']  # tensor [num_comments, hidden]
    feat = hidden_comment.detach().cpu().numpy()

    # Features: concat (parent_emb || child_emb)
    X_edge = np.concatenate([feat[edge_src], feat[edge_dst]], axis=1)
    # Labels: abusive if child comment toxic (1), else 0
    y_edge = hetero['comment'].y.detach().cpu().numpy()[edge_dst]

    X_train, X_test, y_train, y_test = train_test_split(
        X_edge, y_edge, test_size=0.2, stratify=y_edge, random_state=42
    )

    import torch.nn as nn
    class EdgeMLP(nn.Module):
        def __init__(self, in_dim, hidden=128):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(in_dim, hidden), nn.ReLU(), nn.Linear(hidden, 2)
            )
        def forward(self, x):
            return self.net(x)

    in_dim = X_train.shape[1]
    clf = EdgeMLP(in_dim)
    opt = torch.optim.Adam(clf.parameters(), lr=1e-3, weight_decay=1e-4)
    lossf = nn.CrossEntropyLoss()

    Xtr = torch.tensor(X_train, dtype=torch.float); ytr = torch.tensor(y_train, dtype=torch.long)
    Xte = torch.tensor(X_test, dtype=torch.float); yte = torch.tensor(y_test, dtype=torch.long)

    for epoch in range(1, 21):
        clf.train(); opt.zero_grad(); out = clf(Xtr); loss = lossf(out, ytr); loss.backward(); opt.step()
        if epoch%5==0:
            clf.eval(); pred = clf(Xte).argmax(1); acc=(pred==yte).float().mean().item(); print(f'Epoch {epoch} loss {loss.item():.4f} test_acc {acc:.4f}')

    clf.eval(); pred = clf(Xte).argmax(1).numpy()
    print(classification_report(y_test, pred, digits=3))

    # Save report
    import json
    from sklearn.metrics import precision_recall_fscore_support
    p, r, f1, s = precision_recall_fscore_support(y_test, pred, labels=[0,1], zero_division=0)
    edge_report = {
        'labels': ['non_abusive','abusive'],
        'precision': p.tolist(), 'recall': r.tolist(), 'f1': f1.tolist(), 'support': s.tolist()
    }
    Path('artifacts').mkdir(exist_ok=True)
    json.dump(edge_report, open('artifacts/edge_clf_report.json','w'), indent=2)
    print('Wrote artifacts/edge_clf_report.json')

## 11c. Motif Counting and k-core (Gang-up Indicators)

In [ ]:
# Convert to NetworkX and compute motifs + k-core
import networkx as nx
from collections import Counter
from pathlib import Path
import json

# Build a directed graph of comment replies
G = nx.DiGraph()
G.add_nodes_from(range(hetero['comment'].num_nodes))
if 'comment' in hetero.node_types and ('comment','replies_to','comment') in hetero.edge_types:
    eidx = hetero['comment','replies_to','comment'].edge_index.detach().cpu().numpy()
    for u,v in zip(eidx[0], eidx[1]):
        G.add_edge(int(u), int(v))

# Edge mask for abusive-only: child comment is toxic
import numpy as np
abusive_edges = []
if G.number_of_edges() > 0:
    y_np = hetero['comment'].y.detach().cpu().numpy()
    abusive_edges = [(u,v) for u,v in G.edges() if y_np[v] == 1]

# Helper to compute summary stats

def graph_summary(Gd: nx.DiGraph):
    if Gd.number_of_nodes() == 0:
        return {
            'n_nodes': 0, 'n_edges': 0, 'density': 0.0,
            'avg_clustering': 0.0, 'triadic_census': {}, 'kcore_max': 0,
            'reciprocity': 0.0
        }
    und = Gd.to_undirected()
    dens = nx.density(und)
    try:
        avg_clust = nx.average_clustering(und) if und.number_of_edges() > 0 else 0.0
    except Exception:
        avg_clust = 0.0
    try:
        triad = nx.triadic_census(Gd)
    except Exception:
        # Fallback: compute simple count via triangle count on undirected
        tri = sum(nx.triangles(und).values()) // 3 if und.number_of_edges() > 0 else 0
        triad = {'triangles': int(tri)}
    try:
        core_nums = nx.core_number(und)
        kcore_max = int(max(core_nums.values())) if core_nums else 0
    except Exception:
        kcore_max = 0
    try:
        reciprocity = nx.reciprocity(Gd)
        reciprocity = float(reciprocity) if reciprocity is not None else 0.0
    except Exception:
        reciprocity = 0.0
    return {
        'n_nodes': Gd.number_of_nodes(),
        'n_edges': Gd.number_of_edges(),
        'density': float(dens),
        'avg_clustering': float(avg_clust),
        'triadic_census': {k:int(v) for k,v in triad.items()},
        'kcore_max': kcore_max,
        'reciprocity': reciprocity
    }

report = {}
report['all_edges'] = graph_summary(G)

# Abusive-only subgraph
if abusive_edges:
    G_ab = nx.DiGraph()
    G_ab.add_nodes_from(G.nodes())
    G_ab.add_edges_from(abusive_edges)
    report['abusive_only'] = graph_summary(G_ab)
else:
    report['abusive_only'] = {
        'n_nodes': G.number_of_nodes(), 'n_edges': 0, 'density': 0.0,
        'avg_clustering': 0.0, 'triadic_census': {}, 'kcore_max': 0,
        'reciprocity': 0.0
    }

Path('artifacts').mkdir(exist_ok=True)
with open('artifacts/motifs_kcore_report.json','w') as f:
    json.dump(report, f, indent=2)
print('Wrote artifacts/motifs_kcore_report.json')

## 12. Task B: User–User Graph, Communities, Polarization

In [ ]:
# Build user-user graph from reply edges; run communities; compute metrics
import networkx as nx
import json
from pathlib import Path
from collections import defaultdict

# Map comment -> user
comment_user = df.set_index(CID_COL)[USER_COL].to_dict()

# Build directed user-user edges (u -> v if u authored parent, v authored child)
U = nx.DiGraph()
U.add_nodes_from(users)  # users list built earlier
for p, c in zip(r_src, r_dst):
    # Ensure indices refer to valid comment ids
    if p < len(comment_ids) and c < len(comment_ids):
        up = comment_user.get(comment_ids[p])
        uc = comment_user.get(comment_ids[c])
        if up is None or uc is None:
            continue
        if up == uc:
            continue  # skip self-loops
        U.add_edge(up, uc)

# Toxic-only user edges: child comment is toxic
y_np = hetero['comment'].y.detach().cpu().numpy()
U_toxic = nx.DiGraph()
U_toxic.add_nodes_from(users)
for p, c in zip(r_src, r_dst):
    if p < len(comment_ids) and c < len(comment_ids) and y_np[c] == 1:
        up = comment_user.get(comment_ids[p])
        uc = comment_user.get(comment_ids[c])
        if up is None or uc is None or up == uc:
            continue
        U_toxic.add_edge(up, uc)

# Community detection (greedy modularity on undirected projection)
Und = U.to_undirected()
Und_t = U_toxic.to_undirected()

try:
    from networkx.algorithms.community import greedy_modularity_communities
    comms = list(greedy_modularity_communities(Und)) if Und.number_of_edges()>0 else []
    comms_t = list(greedy_modularity_communities(Und_t)) if Und_t.number_of_edges()>0 else []
except Exception:
    comms, comms_t = [], []

# Build partition dicts
part = {}
for i, com in enumerate(comms):
    for u in com:
        part[u] = i
part_t = {}
for i, com in enumerate(comms_t):
    for u in com:
        part_t[u] = i

# Modularity (if possible)
mod = None
mod_t = None
try:
    from networkx.algorithms.community.quality import modularity
    if comms:
        mod = float(modularity(Und, comms))
    if comms_t:
        mod_t = float(modularity(Und_t, comms_t))
except Exception:
    pass

# E-I index: for directed graph, compare inter-community edges to intra-community

def ei_index(Gd: nx.DiGraph, partition: dict):
    if Gd.number_of_edges() == 0:
        return 0.0
    internal = external = 0
    for u, v in Gd.edges():
        cu = partition.get(u, -1)
        cv = partition.get(v, -1)
        if cu == -1 or cv == -1:
            continue
        if cu == cv:
            internal += 1
        else:
            external += 1
    denom = internal + external
    return float((external - internal) / denom) if denom > 0 else 0.0

ei = ei_index(U, part)
ei_t = ei_index(U_toxic, part_t if part_t else part)

# Save partitions and metrics
Path('artifacts').mkdir(exist_ok=True)
part_rows = [(u, part.get(u, -1)) for u in users]
part_t_rows = [(u, part_t.get(u, -1)) for u in users]

import pandas as pd
pd.DataFrame(part_rows, columns=['user','community']).to_csv('artifacts/user_partition.csv', index=False)
pd.DataFrame(part_t_rows, columns=['user','community_toxic']).to_csv('artifacts/user_partition_toxic.csv', index=False)

comm_report = {
    'n_users': int(len(users)),
    'user_edges_all': int(U.number_of_edges()),
    'user_edges_toxic': int(U_toxic.number_of_edges()),
    'n_communities_all': int(len(comms)),
    'n_communities_toxic': int(len(comms_t)),
    'modularity_all': mod,
    'modularity_toxic': mod_t,
    'ei_index_all': ei,
    'ei_index_toxic': ei_t
}
with open('artifacts/communities_report.json','w') as f:
    json.dump(comm_report, f, indent=2)
print('Wrote artifacts/user_partition.csv, user_partition_toxic.csv, communities_report.json')